# Ensemble 
This notebook is used to verify the effectiveness of some **ensemble** techniques, such as blending, stacking, etc. After the core code snippet is verified, the complete process will be written into script file.

In [ ]:
import os
import pickle
from functools import partial

import pandas as pd 
import wandb
from bayes_opt import BayesianOptimization

from metadata import *

from utils.common import rank_mcls_naive
from utils.evaluator import EvaluatorRank

In [ ]:
# Init Wandb run entry
ens = wandb.init(project='Esun',
                 job_type='ensemble')

In [9]:
# Shortcut for accessing submission file of the specified version 
# output = ens.use_artifact(f'lgbm_infer:v66', type='output')
# output_dir = output.download()
# with open(os.path.join(output_dir, 'pred_results', 'dt25.pkl'), 'rb') as f:
#     oof_pred = pickle.load(f)
# oof_pred['y_pred'].to_csv("./submission.csv", index=False)

wandb: Downloading large artifact lgbm_infer:v66, 80.11MB. 1 files... Done. 0:0:0


## *Blending*
Blending uses weighted sum of the predicting probability distributions from different base models, trying to boost the performance by considering each base model's merit.
<div class="alert alert-block alert-danger">
    <p>Implementation in cells below provides me with a playground to quickly see how blending performs. Upon obtaining a satisfactory validation score, formal blending is run using script <code>blend.py</code> to obtain the blended results for submission or next-level ensemble (<i>e.g.</i>, stacking).</p>
</div>

### **Blending Candidates**
|                 |      |      |      |      |     |     |     |
| --------------- | ---- | ---- | ---- | ---- | --- | --- | --- |
| Trained Type    | lgbm | lgmb | lgbm |  | xgb | xgb | xgb |
| Trained Version | 11 | 12 | 16 |  | 3 | 4 | 8 |
| NDCG@3 (dt24)   | 0.72674 | 0.72626 | 0.72726 |  | 0.72670 | 0.72655 | 0.72755 |

### 1. Naive Blending 

In [2]:
def naive_blend(ens, trained_types, trained_versions):
    '''Blend predicting results from multiple base models by summing
    them with equal weights.
    
    Parameter:
        ens: obj, wandb run object
        trained_types: list, meta-model type
        trained_versions: list, meta-model version
    
    Return:
        pk:
        y_true:
        oof_pred_blended:
    '''
    pk, y_true = None, None 
    oof_pred_blended = None
    for t, (i, v) in zip(trained_types, enumerate(trained_versions)):
        output = ens.use_artifact(f'{t}:v{v}', type='output')
        output_dir = output.download()
        with open(os.path.join(output_dir, 'pred_reports/24.pkl'), 'rb') as f:
            oof_pred = pickle.load(f)
        if i == 0:
            oof_pred_blended = oof_pred['y_pred'] / len(trained_versions)
            pk = oof_pred['index']
            y_true = oof_pred['y_true']
        else: oof_pred_blended += oof_pred['y_pred'] / len(trained_versions)
        del output, oof_pred
    return pk, y_true, oof_pred_blended

In [10]:
# Specify versions to blend
blend_candidates = [
    (['lgbm_meta', 'lgbm_meta', 'xgb_meta', 'xgb_meta'], [11, 12, 3, 4]),
    (['lgbm_meta', 'xgb_meta', 'xgb_meta'], [11, 3, 4]),
    (['lgbm_meta', 'xgb_meta', 'xgb_meta'], [12, 3, 4]),
    (['lgbm_meta', 'xgb_meta'], [11, 3]),
    (['lgbm_meta', 'xgb_meta'], [11, 4]),
    (['lgbm_meta', 'xgb_meta'], [12, 3]),
    (['lgbm_meta', 'xgb_meta'], [12, 4]),
    (['lgbm_meta', 'lgbm_meta'], [11, 12]),
    (['xgb_meta', 'xgb_meta'], [3, 4]),
    #------------------------------------#
    (['lgbm_meta', 'xgb_meta'], [16, 8]),
]

In [5]:
# Run naive blending with equal weights
for trained_types, trained_versions in blend_candidates:
    pk, y_true, oof_pred_blended = naive_blend(ens, trained_types, trained_versions)
    oof_rank_blended = rank_mcls_naive(pk, oof_pred_blended)
    evaluator = EvaluatorRank("./data/raw/raw_data.parquet", t_next=24)
    ndcg = evaluator.evaluate(oof_rank_blended)
    print(f"{trained_types} | {trained_versions}")
    print(f"NDCG@3 with blending: {ndcg}!!")
    del pk, y_true, oof_pred_blended, oof_rank_blended, evaluator

wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:17<00:00, 28160.67it/s]


['lgbm_meta', 'lgbm_meta', 'xgb_meta', 'xgb_meta'] | [11, 12, 3, 4]
NDCG@3 with blending: {'NDCG@3': 0.7270940318282572}!!


wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:17<00:00, 27981.29it/s]


['lgbm_meta', 'xgb_meta', 'xgb_meta'] | [11, 3, 4]
NDCG@3 with blending: {'NDCG@3': 0.727134276681944}!!


wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:18<00:00, 27370.62it/s]


...

['lgbm_meta', 'xgb_meta'] | [12, 4]
NDCG@3 with blending: {'NDCG@3': 0.7265462135286991}!!


wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 59887.31it/s]


['lgbm_meta', 'lgbm_meta'] | [11, 12]
NDCG@3 with blending: {'NDCG@3': 0.7269332014177545}!!


wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 62081.52it/s]


['xgb_meta', 'xgb_meta'] | [3, 4]
NDCG@3 with blending: {'NDCG@3': 0.7270662148600788}!!


In [8]:
# Run naive blending with equal weights
for trained_types, trained_versions in blend_candidates[-1:]:
    pk, y_true, oof_pred_blended = naive_blend(ens, trained_types, trained_versions)
    oof_rank_blended = rank_mcls_naive(pk, oof_pred_blended)
    evaluator = EvaluatorRank("./data/raw/raw_data.parquet", t_next=24)
    ndcg = evaluator.evaluate(oof_rank_blended)
    print(f"{trained_types} | {trained_versions}")
    print(f"NDCG@3 with blending: {ndcg}!!")
    del pk, y_true, oof_pred_blended, oof_rank_blended, evaluator

wandb: Downloading large artifact lgbm_meta:v16, 557.89MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v8, 768.16MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 61933.84it/s]


['lgbm_meta', 'xgb_meta'] | [16, 8]
NDCG@3 with blending: {'NDCG@3': 0.7275279599889455}!!


### 2. Bayesian Opt
Instead of blending predicting results with equal weights, **bayesian optimization** is a technique helping us find a better set of blending weight (*i.e.*, coefficient).

In [3]:
def ndcg_fn(ens, trained_types, trained_versions, **coefs):
    '''Return NDCG@3 of the blended model given blending weights.
    
    Parameters:
        ens: obj, wandb run object
        trained_types: list, meta-model type
        trained_versions: list, meta-model version
        coefs: dict, coefficient of each base model considered in 
               blending
    
    Return:
        ndcg_blended: float, NDCG@3 of the blended model 
    '''
    pk = None
    oof_pred_blended = None
    for i, (_, coef) in enumerate(coefs.items()):
        tr_type, tr_ver = trained_types[i], trained_versions[i]
        tr_ver = int("".join([c for c in tr_ver if c.isdigit()]))
        output = ens.use_artifact(f'{tr_type}:v{tr_ver}', type='output')
        output_dir = output.download()
        with open(os.path.join(output_dir, 'pred_reports/24.pkl'), 'rb') as f:
            oof_pred = pickle.load(f)
        if i == 0:
            oof_pred_blended = oof_pred['y_pred'] * coef
            pk = oof_pred['index']
        else: oof_pred_blended += oof_pred['y_pred'] * coef
        del output, oof_pred
    oof_rank_blended = rank_mcls_naive(pk, oof_pred_blended)
    evaluator = EvaluatorRank("./data/raw/raw_data.parquet", t_next=24)
    ndcg = evaluator.evaluate(oof_rank_blended)['NDCG@3']
    
    return ndcg

#### a. Illegal-Focused

In [4]:
# Configure base models to put in bayesian opt blending 
trained_types = ['lgbm' for  _  in range(11)]
trained_versions = [str(v) for v in range(173, 184)]
coefs = {v: (0, 1) for v in trained_versions}

In [5]:
# Run bayesian optimization
ndcg_fn_ = partial(ndcg_fn, ens, trained_types, trained_versions)
optimizer = BayesianOptimization(f=ndcg_fn_, pbounds=coefs, random_state=168)
optimizer.maximize(init_points=50, n_iter=20)

|   iter    |  target   |    173    |    174    |    175    |    176    |    177    |    178    |    179    |    180    |    181    |    182    |    183    |
-------------------------------------------------------------------------------------------------------------------------------------------------------------


wandb: Downloading large artifact lgbm:v173, 207.28MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v174, 207.34MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v175, 270.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v176, 206.91MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v177, 270.71MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v178, 213.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v179, 268.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v180, 233.63MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v181, 229.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v182, 233.66MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v183, 242.98MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:07<00:00, 67779.71it/s]


|  1        |  0.6989   |  0.5978   |  0.5208   |  0.2468   |  0.3375   |  0.8464   |  0.6514   |  0.7334   |  0.5118   |  0.9043   |  0.9863   |  0.7798   |


wandb: Downloading large artifact lgbm:v173, 207.28MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v174, 207.34MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v175, 270.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v176, 206.91MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v177, 270.71MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v178, 213.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v179, 268.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v180, 233.63MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v181, 229.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v182, 233.66MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v183, 242.98MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:07<00:00, 67500.72it/s]


|  2        |  0.6988   |  0.9185   |  0.4017   |  0.7521   |  0.5576   |  0.4627   |  0.9447   |  0.9432   |  0.7974   |  0.4409   |  0.2203   |  0.3619   |


...

|  68       |  0.6994   |  0.0      |  0.0      |  0.0      |  0.0      |  0.6389   |  1.0      |  0.0      |  0.0      |  0.0      |  0.6801   |  0.5666   |


wandb: Downloading large artifact lgbm:v173, 207.28MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v174, 207.34MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v175, 270.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v176, 206.91MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v177, 270.71MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v178, 213.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v179, 268.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v180, 233.63MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v181, 229.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v182, 233.66MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v183, 242.98MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 60190.10it/s]


|  69       |  0.6994   |  0.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.6411   |  0.0      |  0.0      |  0.0      |  0.6252   |  0.5854   |


wandb: Downloading large artifact lgbm:v173, 207.28MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v174, 207.34MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v175, 270.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v176, 206.91MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v177, 270.71MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v178, 213.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v179, 268.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v180, 233.63MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v181, 229.75MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v182, 233.66MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v183, 242.98MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 59713.39it/s]


|  70       |  0.6994   |  0.0      |  0.0      |  0.0      |  0.0      |  0.52     |  1.0      |  0.0      |  0.0      |  0.0      |  0.2782   |  1.0      |


In [7]:
# Obtain the best set of blending coefficients
optimizer.max

{'target': 0.6994350462362194,
 'params': {'173': 0.0,
  '174': 0.0,
  '175': 0.0,
  '176': 0.0,
  '177': 1.0,
  '178': 1.0,
  '179': 0.0,
  '180': 0.0,
  '181': 0.0,
  '182': 0.6709499726648304,
  '183': 0.6827723996942069}}

In [9]:
# Manually blend top4 with equal weights
train_v = [177, 178, 182, 183]
coef_manual = {str(i): 1 for i in a}
ndcg_fn(ens, ['lgbm' for _ in range(4)], train_v, **coef_manual)

wandb: Downloading large artifact lgbm:v177, 270.71MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v178, 213.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v182, 233.66MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v183, 242.98MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 60718.43it/s]


0.6994156694216868

#### b. Final Godset

In [3]:
# Configure base models to put in bayesian opt blending 
trained_types = ['lgbm' for  _  in range(12)]
trained_versions = [str(v) for v in range(184, 196)]
coefs = {v: (0, 1) for v in trained_versions}

In [5]:
# Run bayesian optimization
ndcg_fn_ = partial(ndcg_fn, ens, trained_types, trained_versions)
optimizer = BayesianOptimization(f=ndcg_fn_, pbounds=coefs, random_state=168)
optimizer.maximize(init_points=50, n_iter=20)

|   iter    |  target   |    184    |    185    |    186    |    187    |    188    |    189    |    190    |    191    |    192    |    193    |    194    |    195    |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------


wandb: Downloading large artifact lgbm:v184, 271.94MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v185, 274.33MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v186, 242.67MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v187, 243.74MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v188, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v189, 215.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v190, 275.19MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v191, 234.21MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v192, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v193, 207.59MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v194, 222.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v195, 264.24MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 60846.59it/s]


|  1        |  0.7095   |  0.5978   |  0.5208   |  0.2468   |  0.3375   |  0.8464   |  0.6514   |  0.7334   |  0.5118   |  0.9043   |  0.9863   |  0.7798   |  0.9185   |


wandb: Downloading large artifact lgbm:v184, 271.94MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v185, 274.33MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v186, 242.67MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v187, 243.74MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v188, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v189, 215.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v190, 275.19MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v191, 234.21MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v192, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v193, 207.59MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v194, 222.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v195, 264.24MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:08<00:00, 60292.10it/s]


|  2        |  0.7094   |  0.4017   |  0.7521   |  0.5576   |  0.4627   |  0.9447   |  0.9432   |  0.7974   |  0.4409   |  0.2203   |  0.3619   |  0.4555   |  0.1673   |


...

|  66       |  0.7143   |  0.2827   |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |
|  67       |  0.7143   |  0.2827   |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |


wandb: Downloading large artifact lgbm:v184, 271.94MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v185, 274.33MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v186, 242.67MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v187, 243.74MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v188, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v189, 215.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v190, 275.19MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v191, 234.21MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v192, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v193, 207.59MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v194, 222.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v195, 264.24MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:05<00:00, 88617.94it/s]


|  68       |  0.7147   |  1.0      |  0.1459   |  1.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |  0.0      |  0.0      |
|  69       |  0.7147   |  1.0      |  0.1459   |  1.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |  0.0      |  0.0      |


wandb: Downloading large artifact lgbm:v184, 271.94MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v185, 274.33MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v186, 242.67MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v187, 243.74MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v188, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v189, 215.22MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v190, 275.19MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v191, 234.21MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v192, 235.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v193, 207.59MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v194, 222.93MB. 3 files... Done. 0:0:0
wandb: Downloading large artifact lgbm:v195, 264.24MB. 3 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:05<00:00, 89648.19it/s]

|  70       |  0.7146   |  1.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.1806   |  1.0      |  0.0      |  0.0      |  0.0      |  0.0      |  0.0      |


In [6]:
# Obtain the best set of blending coefficients
optimizer.max

{'target': 0.7148598510778752,
 'params': {'184': 1.0,
  '185': 0.0,
  '186': 1.0,
  '187': 0.0,
  '188': 0.0,
  '189': 0.0,
  '190': 1.0,
  '191': 0.0,
  '192': 0.0,
  '193': 0.0,
  '194': 0.0,
  '195': 0.0}}

#### c. Meta-models

In [4]:
# Configure base models to put in bayesian opt blending 
meta_types = ['lgbm_meta' for  _  in range(6)] + ['xgb_meta' for _ in range(5)]
meta_versions = [f'l{v}' for v in [11, 12, 16, 17, 18, 19]] + [f'x{v}' for v in [3, 4, 8, 9, 10]]
coefs = {v: (0, 1) for v in meta_versions}

In [5]:
# Run bayesian optimization
ndcg_fn_ = partial(ndcg_fn, ens, meta_types, meta_versions)
optimizer = BayesianOptimization(f=ndcg_fn_, pbounds=coefs, random_state=168)
optimizer.maximize(init_points=50, n_iter=20)

|   iter    |  target   |    l11    |    l12    |    l16    |    l17    |    l18    |    l19    |    x10    |    x3     |    x4     |    x8     |    x9     |
-------------------------------------------------------------------------------------------------------------------------------------------------------------


wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v16, 557.89MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v17, 558.15MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v18, 554.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v19, 552.47MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v8, 768.16MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v9, 759.90MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v10, 766.76MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:05<00:00, 87190.26it/s]


|  1        |  0.7278   |  0.5978   |  0.5208   |  0.2468   |  0.3375   |  0.8464   |  0.6514   |  0.7334   |  0.5118   |  0.9043   |  0.9863   |  0.7798   |


wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v16, 557.89MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v17, 558.15MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v18, 554.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v19, 552.47MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v8, 768.16MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v9, 759.90MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v10, 766.76MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:05<00:00, 88976.27it/s]


|  2        |  0.7277   |  0.9185   |  0.4017   |  0.7521   |  0.5576   |  0.4627   |  0.9447   |  0.9432   |  0.7974   |  0.4409   |  0.2203   |  0.3619   |


...

|  68       |  0.728    |  0.0      |  0.0      |  1.0      |  0.0      |  1.0      |  0.8706   |  0.0      |  0.1375   |  0.3295   |  0.0      |  1.0      |


wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v16, 557.89MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v17, 558.15MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v18, 554.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v19, 552.47MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v8, 768.16MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v9, 759.90MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v10, 766.76MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:05<00:00, 88342.49it/s]


|  69       |  0.7281   |  0.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |  0.3964   |  0.0      |  0.5981   |


wandb: Downloading large artifact lgbm_meta:v11, 546.02MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v12, 557.04MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v16, 557.89MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v17, 558.15MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v18, 554.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact lgbm_meta:v19, 552.47MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v3, 756.43MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v4, 765.65MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v8, 768.16MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v9, 759.90MB. 7 files... Done. 0:0:0
wandb: Downloading large artifact xgb_meta:v10, 766.76MB. 7 files... Done. 0:0:0
100%|██████████| 500000/500000 [00:05<00:00, 88217.60it/s]


|  70       |  0.7281   |  0.0      |  0.0      |  0.0      |  0.0      |  1.0      |  0.0      |  0.0      |  0.0      |  0.8194   |  0.0      |  0.1326   |


In [6]:
# Obtain the best set of blending coefficients
optimizer.max

{'target': 0.7280846333489985,
 'params': {'l11': 0.0,
  'l12': 0.0,
  'l16': 0.0,
  'l17': 0.0,
  'l18': 1.0,
  'l19': 0.0,
  'x10': 0.0,
  'x3': 0.0,
  'x4': 0.39640406184048566,
  'x8': 0.0,
  'x9': 0.5980503849892724}}

## *NN Stacker*
To diversify level-2 meta-models, I decide to put simple feed-foward NN into meta-model pool, and see how well it can perform compared with tree-based stackers.

```
Given X, y 
    X: pd.DataFrame, predicitng results of base models 
    y: ndarray, groundtruths
    
Generate simple Dataset
For all epoch in EPOCHES
    For all batches in Dataloader
        Run simple forward passing and backpropagation using FFNN
    Until running through all data samples
Until convergence
```

In [6]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
class FFNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout, 
                 act, output_dim=16):
        super(FFNN, self).__init__()
        
        # Network parameters
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.act = act
        self.output_dim = output_dim
        self.dropout = dropout        
        
        # Model blocks
        self.hidden = nn.Linear(self.input_dim, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)
        if act == 'relu':
            self.hidden_act = F.relu
        elif act == 'selu':
            self.hidden_act = F.selu
        elif act == 'sigmoid':
            self.hidden_act = F.sigmoid
        elif act == 'tanh':
            self.hidden_act = F.tanh
        self.dropout = nn.Dropout(p=self.dropout)
        
    def forward(self, x):
        x = self.hidden_act(self.hidden(x))
        x = self.dropout(x)
        
        logits = self.output(x)
        
        return logits 

In [4]:
def get_pred_df(oof_pred):
    df = pd.DataFrame()
    df.index = oof_pred['index']
    df['gts'] = oof_pred['y_true']
    cols = [f'shop_tag{i}' for i in LEG_SHOP_TAGS]
    df[cols] = oof_pred['y_pred']
    
    return df 